In [1]:
import actorcritic as ac
import run_experiment as expt
import environment as gw

import numpy as np
import torch

In [2]:
ep = expt.basic_env_params()
env = gw.GridWorld(rows=ep.rows, cols=ep.columns, env_type=ep.env_type,
                           rewards={ep.reward_location: ep.reward_mag},
                           step_penalization=ep.penalty,
                           rho=ep.rho,
                           actionlist=ep.actionlist,
                           rewarded_action=ep.rewarded_action)
agent_parameters = expt.basic_agent_params(env)
agent_parameters.input_dims = env.nstates
agent_parameters.hidden_types = ['linear', 'linear', 'linear', 'linear', 'linear', 'linear']
agent_parameters.hidden_dims = [1000, 2000, 2000, 2000, 1000, 1000]

print(agent_parameters.__dict__)

{'load_model': False, 'load_dir': '', 'architecture': 'B', 'input_dims': 400, 'action_dims': 4, 'hidden_types': ['linear', 'linear', 'linear', 'linear', 'linear', 'linear'], 'hidden_dims': [1000, 2000, 2000, 2000, 1000, 1000], 'freeze_w': False, 'rfsize': 5, 'gamma': 0.98, 'eta': 0.0005, 'use_EC': False}


In [3]:
agent = ac.ActorCritic(agent_parameters.__dict__)

In [4]:
NUM_TRIALS = 100
NUM_EVENTS = 20
for trial in range(NUM_TRIALS):
    # reset environment, choose new starting location for agent
    env.resetEnvironment()
    # clear hidden layer cache if using lstm or gru cells
    agent.reinit_hid()
    reward_sum = 0
    
    for event in range(NUM_EVENTS):
        # get state observation
        observation = torch.Tensor(np.expand_dims(env.get_observation(), axis=0))
        # pass observation through network
        if agent.use_SR:
            policy_, value_, phi_, psi_ = agent(observation)
            agent.saved_phi.append(phi_)
            agent.saved_psi.append(psi_)
        else:
            policy_, value_, psi_ = agent(observation)

        # linear activity
        choice = agent.select_action(policy_, value_)
        
        # select action from policy
        action = env.action_list[choice][0]

        # take a step in the environment
        s_1d, reward, isdone = env.move(action)

        agent.saved_rewards.append(reward)
        reward_sum += reward

        if isdone:
            if agent.use_SR:
                agent.saved_phi.append(phi_)
                agent.saved_psi.append(psi_)
            encountered_reward = True
            break

    if agent.use_SR:
        p_loss, v_loss, psi_loss = agent.finish_trial()
    else:
        p_loss, v_loss = agent.finish_trial()

    if trial == 0 or trial%print_freq==0 or trial == NUM_TRIALS - 1:
        print(f"{trial}: {self.reward_sum} ({time.time() - t}s / Running Av: {running_rwdavg}")
        t = time.time()

valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
valueshape torch.Size([1, 1, 1, 1])
torch.Size([1, 1, 1, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 1, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 1, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 1, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 1, 1]) torch.Size([1, 1, 1])
torch.Size([1, 1, 1, 1]) torch.Size([1, 1, 1]

/home/annik/Dropbox/LINC Lab Documents/Code/MEMRL/episodic_control/actorcritic/__init__.py:281: UserWarning: Using a target size (torch.Size([1, 1, 1])) that is different to the input size (torch.Size([1, 1, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  value_losses.append(F.smooth_l1_loss(value, return_bs))


RuntimeError: zero-dimensional tensor (at position 0) cannot be concatenated